In [1]:
import datasets
from subset_active_learning.subset_selection import select, preprocess
import wandb

/home/glai/anaconda3/envs/subset/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
processed_ds = preprocess.preprocess_sst2("google/electra-small-discriminator")

No config specified, defaulting to: sst/default
Reusing dataset sst (/home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)
100%|██████████| 3/3 [00:00<00:00, 809.55it/s]
Loading cached processed dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-7b23d5d1250b3cef.arrow
Loading cached processed dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-6a94e7b0eb5aeeeb.arrow
Loading cached processed dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-f1806eefe95f0de4.arrow
Loading cached processed dataset at /home/glai/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-e283858a38c31b48.arrow
Loading cache

In [3]:
class BatchSizeComparisonRun: 
    def __init__(self, train_ds: datasets.Dataset, valid_ds: datasets.Dataset, test_ds: datasets.Dataset):
        self.train_ds,self.valid_ds,self.test_ds = train_ds,valid_ds,test_ds

    def _one_run(self, wandb_tag: str, config: select.SubsetTrainingArguments):
        wandb_run = wandb.init(project="subset-search-gpu-opt", entity="johnny-gary", tags=[wandb_tag])
        wandb.log({"batch_size": config.batch_size})
        subset_trainer = select.SubsetTrainer(
            params=config, valid_ds=self.valid_ds, test_ds=self.test_ds
        )
        subset_trainer.train_one_step(subset=self.train_ds, calculate_test_accuracy=True)
        wandb_run.finish()

    def run_comparison(self, small_batch_config: select.SubsetTrainingArguments, large_batch_config: select.SubsetTrainingArguments): 
        """
        - train small batch size until early stopping
        - train large batch size until early stopping
        """
        self._one_run(wandb_tag=f"small_batch_{small_batch_config.batch_size}", config=small_batch_config)
        self._one_run(wandb_tag=f"large_batch_{large_batch_config.batch_size}", config=large_batch_config)
        pass

In [8]:
INCREASE_FACTOR = 4
small_batch_config = select.SubsetTrainingArguments(batch_size=8, learning_rate=1e-5)
large_batch_config = select.SubsetTrainingArguments(batch_size=small_batch_config.batch_size*INCREASE_FACTOR, learning_rate=small_batch_config.learning_rate*INCREASE_FACTOR)

In [9]:
for seed in range(43, 47):
    train_ds = processed_ds["train"].shuffle(seed=seed).select(range(100))
    batch_size_comparison = BatchSizeComparisonRun(train_ds=train_ds, valid_ds=processed_ds["validation"], test_ds=processed_ds["test"])
    batch_size_comparison.run_comparison(small_batch_config=small_batch_config, large_batch_config=large_batch_config)

Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

batch_size,▁
loss,███████▇▇█▇▅▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
sst2_final_test:accuracy,▁
sst2_final_valid:accuracy,▁
sst:val_acc,▁█████
batch_size,8
loss,0.0034
sst2_final_test:accuracy,0.66878
sst2_final_valid:accuracy,0.6594
sst:val_acc,0.6594


Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

batch_size,▁
loss,███▇▇▅▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
sst2_final_test:accuracy,▁
sst2_final_valid:accuracy,▁
sst:val_acc,▁▇█▃▆
batch_size,32
loss,0.00045
sst2_final_test:accuracy,0.65656
sst2_final_valid:accuracy,0.67575
sst:val_acc,0.67575


Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

...

batch_size,▁
loss,███▇▇▆▇▇▇▆▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
sst2_final_test:accuracy,▁
sst2_final_valid:accuracy,▁
sst:val_acc,█▅▁
batch_size,32
loss,0.0011
sst2_final_test:accuracy,0.65882
sst2_final_valid:accuracy,0.65213
sst:val_acc,0.65213


Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

batch_size,▁
loss,███████▇▇▆▅▄▄▃▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
sst2_final_test:accuracy,▁
sst2_final_valid:accuracy,▁
sst:val_acc,▁▆██▇█
batch_size,8
loss,0.00343
sst2_final_test:accuracy,0.67149
sst2_final_valid:accuracy,0.66757
sst:val_acc,0.66757


Some weights of the model checkpoint at google/electra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-small-discriminator and are newly initialized: ['classifier

batch_size,▁
loss,█████▇▇▄▃▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
sst2_final_test:accuracy,▁
sst2_final_valid:accuracy,▁
sst:val_acc,▁█▇█
batch_size,32
loss,0.00076
sst2_final_test:accuracy,0.67014
sst2_final_valid:accuracy,0.67484
sst:val_acc,0.67484
